In [1]:
from pyiron_workflow_lammps.engine import LammpsEngine
from pyiron_workflow_atomistics.dataclass_storage import CalcInputMinimize, CalcInputStatic
from pyiron_workflow.workflow import Workflow
from pyiron_workflow_atomistics.bulk import optimise_cubic_lattice_parameter
from pyiron_workflow_atomistics.structure_manipulator.tools import create_supercell_with_min_dimensions
from pyiron_workflow_atomistics.gb.gb_study import pure_gb_study
from pyiron_workflow_atomistics.gb.dataclass_storage import CleaveGBStructureInput, PlotCleaveInput
from pyiron_workflow_atomistics.featurisers import distanceMatrixSiteFeaturiser, voronoiSiteFeaturiser
from pyiron_workflow_atomistics.gb.gb_code.searcher import get_gb_code_df, get_gb_code_df_with_structures
from pyiron_workflow_atomistics.structure_manipulator.tools import substitutional_swap_one_site

import pyiron_workflow as pwf
import os
import numpy as np
from ase.build import bulk

%load_ext autoreload
%autoreload 2

In [2]:
inp = CalcInputMinimize()
inp.relax_cell = False
Engine_eam = LammpsEngine(EngineInput = inp)

Engine_eam.working_directory = "solution_energy"
Engine_eam.lammps_log_filepath = "minimize.log"
# Adjust this line to match your LAMMPS installation
# If you are using conda-lammps or a LAMMPS binary on your system, you can use directly:
Engine_eam.command = "lmp -in in.lmp -log minimize.log"
Engine_eam.input_script_pair_style = "eam/fs"
Engine_eam.path_to_model = os.getcwd() + "/Al-Fe.eam.fs"

In [ ]:
Fe_bulk = bulk("Fe", a=2.828, cubic=True)

wf = Workflow(Engine_eam.working_directory, delete_existing_savefiles=True)
wf.opt_cubic_cell = optimise_cubic_lattice_parameter(
    structure=Fe_bulk,
    name="Fe",
    crystalstructure="bcc",
    calculation_engine=Engine_eam,
    parent_working_directory="opt_cubic_cell",
    rattle=0.1,
    strain_range=(-0.02, 0.02),
    num_points=6,
)
wf.supercell = create_supercell_with_min_dimensions(
    base_structure=wf.opt_cubic_cell.outputs.equil_struct,
    min_dimensions=[12, 12, 12],
)
from pyiron_workflow_atomistics.structure_manipulator.tools import substitutional_swap_one_site
wf.supercell_with_1sol = substitutional_swap_one_site(
    base_structure=wf.supercell,
    defect_site=0,
    new_symbol="Al",
)
from pyiron_workflow_atomistics.calculator import calculate_structure_node

Engine_eam = LammpsEngine(EngineInput = inp)
Engine_eam.working_directory = "solution_energy/solution_energy_bulk"
Engine_eam.lammps_log_filepath = "minimize.log"
# Adjust this line to match your LAMMPS installation
# If you are using conda-lammps or a LAMMPS binary on your system, you can use directly:
Engine_eam.command = "lmp -in in.lmp -log minimize.log"
Engine_eam.input_script_pair_style = "eam/fs"
Engine_eam.path_to_model = os.getcwd() + "/Al-Fe.eam.fs"

wf.calc_supercell_energy = calculate_structure_node(
    structure=wf.supercell,
    calculation_engine=Engine_eam,
)

Engine_eam = LammpsEngine(EngineInput = inp)
Engine_eam.working_directory = "solution_energy/solution_energy_solute"
Engine_eam.lammps_log_filepath = "minimize.log"
Engine_eam.command = "lmp -in in.lmp -log minimize.log"
Engine_eam.input_script_pair_style = "eam/fs"
Engine_eam.path_to_model = os.getcwd() + "/Al-Fe.eam.fs"

wf.calc_supercell_with_1sol_energy = calculate_structure_node(
    structure=wf.supercell_with_1sol,
    calculation_engine=Engine_eam,
)
@pwf.as_function_node("solution_energy")
def calculate_soln_energy(bulk_structure_energy, soln_structure_energy):
    return soln_structure_energy - bulk_structure_energy
wf.soln_energy = calculate_soln_energy(
    bulk_structure_energy=wf.calc_supercell_energy.outputs.calc_output.final_energy,
    soln_structure_energy=wf.calc_supercell_with_1sol_energy.outputs.calc_output.final_energy,
)
wf.run()

/root/miniconda3/envs/pyiron_pypi_workflow_lammps/lib/python3.12/site-packages/pyiron_workflow_lammps/engine.py:113: RuntimeWarning: 'mode' already set to 'minimize' and also implied by EngineInput; no change.
  warnings.warn(


current mode  minimize
current mode  minimize
current mode  minimize


{'opt_cubic_cell__a0': np.float64(2.8554036006730863),
 'opt_cubic_cell__B': np.float64(178.2689296133779),
 'opt_cubic_cell__equil_energy_per_atom': np.float64(-4.012989682254272),
 'opt_cubic_cell__equil_volume_per_atom': np.float64(11.640523523888817),
 'opt_cubic_cell__volumes': [np.float64(21.287097162601995),
  np.float64(21.81268105404494),
  np.float64(22.346845717919777),
  np.float64(22.889660634165235),
  np.float64(23.441195282719992),
  np.float64(24.001519143522813)],
 'opt_cubic_cell__structures': [Atoms(symbols='Fe2', pbc=True, cell=[[2.77144, 1.6970175625144705e-16, 1.6970175625144705e-16], [0.0, 2.77144, 1.6970175625144705e-16], [0.0, 0.0, 2.77144]]),
  Atoms(symbols='Fe2', pbc=True, cell=[[2.794064, 1.7108707671064253e-16, 1.7108707671064253e-16], [0.0, 2.794064, 1.7108707671064253e-16], [0.0, 0.0, 2.794064]]),
  Atoms(symbols='Fe2', pbc=True, cell=[[2.816688, 1.72472397169838e-16, 1.72472397169838e-16], [0.0, 2.816688, 1.72472397169838e-16], [0.0, 0.0, 2.816688]]),


In [ ]:
print(f"The solution energy of Al in Fe BCC is {wf.soln_energy.outputs.solution_energy.value:.3f} eV")
print(f"The total energy of the bulk (128 Fe) structure is {wf.calc_supercell_energy.outputs.calc_output.value.final_energy:.3f} eV")
print(f"The total energy of the solute (127 Fe + 1 Al) structure is {wf.calc_supercell_with_1sol_energy.outputs.calc_output.value.final_energy:.3f} eV")

The solution energy is -0.369 eV
The total energy of the bulk (128 Fe) structure is -1003.246 eV
The total energy of the solute (127 Fe + 1 Al) structure is -1003.615 eV
